In [1]:
import plotly.express as px
import numpy as np
from sklearn.decomposition import PCA
from faiss_search import faiss_topk_labels,compute_topk_accuracy
import matplotlib
import matplotlib.pyplot as plt


In [2]:
label_list = ['丝瓜', '你', '关门', '凳子', '厕所', '口渴', '吃',\
               '喝', '嘴巴', '外卖', '头疼', '家人', '小刀', '帮助',\
                  '平静', '心情', '怎样', '感觉', '愿意', '我', '手机',\
                    '找', '把', '护士', '拿', '换药', '放在', '是', '有',\
                          '朋友', '橙汁', '毛巾', '汤圆', '漂亮', '热水', \
                            '猪肉', '玩', '电脑', '看', '碗', '穿', '篮球',\
                                  '米饭', '给', '脸盆', '菠萝', '葱花', '蒜泥',\
                                      '衣服', '豆腐', '软糖', '醋', '钢琴', '问题',\
                                          '需要', '青菜', '面条', '音乐', '预约', '香肠', '鱼块']

In [3]:
def load_npz_vis(path="Duin_vit_embeddings.npz"):
    data = np.load(path, allow_pickle=True)
    return data["chars"], data["embeddings"], dict(data["meta"])

def load_npz_semantic(path):
    data = np.load(path, allow_pickle=True)
    words = data['words']
    emb_cls = data['emb_cls']
    emb_mean = data['emb_mean']
    emb_max = data['emb_max']
    emb_weighted = data['emb_weighted']
    emb_mixed = data['emb_mixed']
    return words, emb_cls, emb_mean, emb_max, emb_weighted, emb_mixed

In [4]:
def load_test_embeddings(path):
    data = np.load(path, allow_pickle=True)
    labels = data[:,-1]
    #把label的元素转化为int
    labels = labels.astype(int)
    labels=[label_list[i] for i in labels]
    labels = np.array(labels)
    embeddings = data[:,0:-1]
    return labels, embeddings

In [5]:
# 读取GT语义embeddings和模型输出的语义embeddigs
words, emb_cls, emb_mean, emb_max, emb_weighted, emb_mixed = \
    load_npz_semantic('../GT_embeddings/61words/Duin_bert_embeddings.npz')

GT_labels_semantic = words
GT_embeddings_semantic = emb_mean
test_labels_semantic, test_embeddings_semantic = load_test_embeddings('../output_embeddings/Semantic_Doublelayer_(512-256)_lr1e-4/test_embeddings_epoch_200_2')
print('语义数据格式')
print('GT_labels_semantic:', GT_labels_semantic.shape)
print('GT_embeddings_semantic:', GT_embeddings_semantic.shape)
print('GT_embeddings_semantic Average:', np.mean(GT_embeddings_semantic))
print('test_labels_semantic:', test_labels_semantic.shape)
print('test_embeddings_semantic:', test_embeddings_semantic.shape)
print('test_embeddings_semantic Average:', np.mean(test_embeddings_semantic))

语义数据格式
GT_labels_semantic: (61,)
GT_embeddings_semantic: (61, 768)
GT_embeddings_semantic Average: -0.00010871876
test_labels_semantic: (329,)
test_embeddings_semantic: (329, 768)
test_embeddings_semantic Average: -5.5004056519686606e-05


In [6]:
# 归一化
GT_embeddings_semantic = GT_embeddings_semantic / np.linalg.norm(GT_embeddings_semantic, axis=1, keepdims=True)
test_embeddings_semantic = test_embeddings_semantic / np.linalg.norm(test_embeddings_semantic, axis=1, keepdims=True)
print('归一化后')
print('GT_embeddings_semantic Average:', np.mean(GT_embeddings_semantic))
print('test_embeddings_semantic Average:', np.mean(test_embeddings_semantic))

归一化后
GT_embeddings_semantic Average: -0.00010871877
test_embeddings_semantic Average: -5.50040562025555e-05


In [7]:
topk_labels, topk_probs = faiss_topk_labels(
    GT_embeddings_semantic,   # (61, 768)
    GT_labels_semantic,       # 长度61
    test_embeddings_semantic, # (329, 768)
    topk=10,
    use_gpu=False,
    normalize=True
)


: 

In [ ]:
sample = 1
print(topk_labels[sample])   # 样本0的前10个预测标签
print(topk_probs[sample])    # 对应的softmax概率
print(test_labels_semantic[sample])

In [ ]:
accs = compute_topk_accuracy(topk_labels, test_labels_semantic, ks=(1,3,5,10))
print(accs)


NameError: name 'compute_topk_accuracy' is not defined